In [8]:
import cv2
import torch
from ultralytics import YOLO

import ultralytics.utils.loss as loss_mod
import torch
import torch.nn as nn

# 1. Tạo stub DFLoss để torch.load không lỗi
class DFLoss(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__()
    def forward(self, preds, targets, *args, **kwargs):
        return torch.tensor(0.0, device=preds.device)

loss_mod.DFLoss = DFLoss
# 1. Thiết lập device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# 2. Load YOLO model
model = YOLO('epoch201.pt')
model.model.fuse()
model.to(device)
if device == 'cuda':
    model.model.half()

# 3. Mở video input và khởi tạo writer
video_path = '7570900295992398706.mp4'
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise IOError(f"Không mở được video {video_path}")

fps = cap.get(cv2.CAP_PROP_FPS)
w   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output1.mp4', fourcc, fps, (w, h))
if not out.isOpened():
    raise IOError("Không mở được VideoWriter")

# 4. Màu cho các class (có thể mở rộng tuỳ số class)
colors = [(255,0,0), (0,255,0), (0,0,255), (255,255,0), (0,255,255)]

# 5. Vòng lặp đọc, infer và hiển thị
with torch.no_grad():
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Hết video hoặc lỗi đọc frame")
            break

        # Resize nếu cần (mặc định giữ nguyên kích thước)
        # frame = cv2.resize(frame, (w, h))

        # Inference
        results = model.predict(
            source=frame,
            device=device,
            half=(device=='cuda'),
            conf=0.25,
            verbose=False
        )[0]

        # Vẽ bounding box và label
        for box in results.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            cls_id         = int(box.cls[0])
            conf           = float(box.conf[0])
            color          = colors[cls_id % len(colors)]
            label          = f"{model.names[cls_id]} {conf:.2f}"

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(
                frame, label, (x1, y1 - 6),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1
            )

        # Hiển thị khung hình
        cv2.imshow('YOLO Inference', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Ngắt bởi người dùng")
            break

        # Ghi vào file output
        out.write(frame)

# 6. Giải phóng tài nguyên
cap.release()
out.release()
cv2.destroyAllWindows()

print("Hoàn tất inference và hiển thị.")


Using device: cuda


Model summary (fused): 168 layers, 11127519 parameters, 0 gradients, 28.4 GFLOPs


Hết video hoặc lỗi đọc frame
Hoàn tất inference và hiển thị.
